### This Notebook show the flow of calls to use the SQL Agent and enable a quick test
* This NB only covers **Text2SQL**, not the query explanation

In [1]:
from langchain_community.embeddings import OCIGenAIEmbeddings

# only for showing the table of rows in this NB
import pandas as pd
from decimal import Decimal

from database_manager import DatabaseManager
from llm_manager import LLMManager
from schema_manager_23ai import SchemaManager23AI
from core_functions import generate_sql_with_models
from utils import get_console_logger
from prompt_template import PROMPT_TEMPLATE

from config import (
    CONNECT_ARGS,
    MODEL_LIST,
    ENDPOINT,
    TEMPERATURE,
    DEBUG,
    EMBED_MODEL_NAME,
)
from config_private import COMPARTMENT_OCID

#### Setup 

In [2]:
logger = get_console_logger()

db_manager = DatabaseManager(CONNECT_ARGS, logger)
llm_manager = LLMManager(MODEL_LIST, ENDPOINT, COMPARTMENT_OCID, TEMPERATURE, logger)

embed_model = OCIGenAIEmbeddings(
    model_id=EMBED_MODEL_NAME,
    service_endpoint=ENDPOINT,
    compartment_id=COMPARTMENT_OCID,
)

logger.info("")
logger.info("Loading Schema Manager...")

schema_manager = SchemaManager23AI(db_manager, llm_manager, embed_model, logger)

2024-09-19 13:11:18,167 - Connecting to the Database...
2024-09-19 13:11:18,179 - Connected, DB engine created...
2024-09-19 13:11:18,180 - LLMManager: Initialising the list of models...
2024-09-19 13:11:18,180 - Model: meta.llama-3-70b-instruct
2024-09-19 13:11:18,344 - Model: cohere.command-r-plus
2024-09-19 13:11:18,474 - 
2024-09-19 13:11:18,475 - Loading Schema Manager...


#### The user request

In [3]:
user_request = "List the top 10 sales by total amount, with product name, customer name, country name for sales in Europe"

#### Generate the schema for the request

In [9]:
%%time
RESTRICTED_SCHEMA = schema_manager.get_restricted_schema(user_request)

2024-09-19 13:14:05,196 - Identifying relevant tables for query...
2024-09-19 13:14:05,198 - - COUNTRIES
2024-09-19 13:14:05,199 - - SALES
2024-09-19 13:14:05,202 - - PRODUCTS
2024-09-19 13:14:05,204 - - LOCATIONS
2024-09-19 13:14:05,206 - - HR_COUNTRIES
2024-09-19 13:14:05,214 - - CHANNELS


CPU times: user 85.9 ms, sys: 13.2 ms, total: 99 ms
Wall time: 1.49 s


#### Generate the SQL

In [10]:
%%time
sql_query = generate_sql_with_models(
        user_request, RESTRICTED_SCHEMA, db_manager, llm_manager, PROMPT_TEMPLATE
)

CPU times: user 17.6 ms, sys: 3.85 ms, total: 21.4 ms
Wall time: 2.84 s


In [6]:
print(sql_query)

SELECT 
    p.prod_name, 
    c.cust_first_name || ' ' || c.cust_last_name AS customer_name, 
    co.country_name, 
    s.amount_sold
FROM 
    sales s
JOIN 
    products p ON s.prod_id = p.prod_id
JOIN 
    customers c ON s.cust_id = c.cust_id
JOIN 
    countries co ON c.country_id = co.country_id
WHERE 
    co.country_region = 'Europe'
ORDER BY 
    s.amount_sold DESC
FETCH FIRST 10 ROWS ONLY


#### Test the SQL query

In [7]:
rows = db_manager.execute_sql(sql_query)

2024-09-19 13:11:23,771 - Found 10 rows..


In [8]:
# use Pandas Dataframe for quick display
df = pd.DataFrame(rows)

df

,amount_sold,country_name,customer_name,prod_name
0,1782.72,United Kingdom,Zoe Ballanger,Envoy Ambassador
1,1782.72,United Kingdom,Goddard Remler,Envoy Ambassador
2,1782.72,Italy,Desma Janssen,Envoy Ambassador
3,1782.72,Germany,Hannah Gutierrez,Envoy Ambassador
4,1782.72,France,Belle East,Envoy Ambassador
5,1782.72,Germany,Biddy Titus,Envoy Ambassador
6,1782.72,United Kingdom,Regina Ogletree,Envoy Ambassador
7,1782.72,Germany,Riva Dunbar,Envoy Ambassador
8,1782.72,Germany,Isabel Kitchens,Envoy Ambassador
9,1782.72,Germany,Fran Keeting,Envoy Ambassador
